In [ ]:
# -*- encoding=utf-8 -*-
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from tensorflow.python.ops import math_ops
import keras

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf

#进行配置，使用30%的GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1
session = tf.Session(config=config)

# 设置session
KTF.set_session(session)

path = './data/'
batchsize=64

import h5py

batchsize = 64
image_size = (299,299)
lambda_func = inception_v3.preprocess_input

width = image_size[0]
height = image_size[1]
input_tensor = Input((height, width, 3))
x = input_tensor
if lambda_func:
    x = Lambda(lambda_func)(x)

base_model = InceptionV3(input_tensor=x, weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(30, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


# 先fine-tune全连接层
for layer in base_model.layers[:]:
    layer.trainable = False


gen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)

gen2 = ImageDataGenerator(rescale=1./255)

train_generator = gen.flow_from_directory(path+"image_train", image_size, shuffle=False,
                                          batch_size=batchsize,class_mode='categorical')
validation_generator = gen2.flow_from_directory(path+"image_val", image_size, shuffle=False,
                                         batch_size=batchsize,class_mode='categorical')


# train = model.predict_generator(train_generator,100)
# test = model.predict_generator(test_generator)

# log_loss
def log_loss(y_true, y_pred):
    epsilon = 1e-7
    predictions = math_ops.to_float(y_pred)
    labels = math_ops.to_float(y_true)
    predictions.get_shape().assert_is_compatible_with(labels.get_shape())
    return -(math_ops.reduce_sum(math_ops.multiply(labels,math_ops.log(predictions)+epsilon)))/batchsize

model.compile(optimizer=keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              loss='categorical_crossentropy',
              metrics=[log_loss])

model.fit_generator(
            train_generator,
            samples_per_epoch=train_generator.samples,
            nb_epoch=50,
            validation_data=validation_generator,
            nb_val_samples=validation_generator.samples/batchsize)


# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.1, momentum=0.9), loss='categorical_crossentropy',metrics=[log_loss])

model.fit_generator(
            train_generator,
            samples_per_epoch=train_generator.samples,
            nb_epoch=50,
            validation_data=validation_generator,
            nb_val_samples=validation_generator.samples/batchsize)


model.save_weights('model/incepV3_e50.h5')

Using TensorFlow backend.


In [31]:
def load_model_and_finetune(MODEL,image_size,model_path,new_model_path,lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(30, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # 读取参数
    model.load_weights(model_path)
    
    
    for layer in model.layers[:248]:
        layer.trainable = False
    for layer in model.layers[248:]:
        layer.trainable = False

    gen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True)

    gen2 = ImageDataGenerator(rescale=1./255)

    train_generator = gen.flow_from_directory(path+"image_train", image_size, shuffle=False,
                                              batch_size=batchsize,class_mode='categorical')
    validation_generator = gen2.flow_from_directory(path+"image_val", image_size, shuffle=False,
                                             batch_size=batchsize,class_mode='categorical')
    
    from keras.optimizers import SGD
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=[log_loss])

    model.fit_generator(
                train_generator,
                samples_per_epoch=train_generator.samples,
                nb_epoch=50,
                validation_data=validation_generator,
                nb_val_samples=validation_generator.samples/batchsize)


    model.save_weights(model_path)
    

load_model_and_finetune(InceptionV3,(299,299),'model/incepV3_e50.h5','model/incepV3_248.h5')

Found 4260 images belonging to 30 classes.
Found 1080 images belonging to 30 classes.


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:52: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=66, epochs=50, validation_steps=16)`


Epoch 1/50
 7/66 [==>...........................] - ETA: 1:21 - loss: 3.2807 - log_loss: 3.2807

KeyboardInterrupt: 

In [33]:
# 直接通过 fine-tune 后的model来获取结果
def write_gap_finetune(MODEL, image_size, model_path, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(30, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # 读取参数
    model.load_weights(model_path)

    gen = ImageDataGenerator()
#     train_generator = gen.flow_from_directory(path+"image", image_size, shuffle=False,
#                                               batch_size=16)
    test_generator = gen.flow_from_directory(path+"test_A", image_size, shuffle=False,
                                             batch_size=16, class_mode=None)

#     train = model.predict_generator(train_generator)
    test = model.predict_generator(test_generator)
#     with h5py.File("./model/gap_finetune_%s.h5"%MODEL.func_name) as h:
#         h.create_dataset("train", data=train)
#         h.create_dataset("test", data=test)
#         h.create_dataset("label", data=train_generator.classes)
    return test


result = write_gap_finetune(InceptionV3,(299,299),'model/incepV3_e50.h5',inception_v3.preprocess_input)

Found 3000 images belonging to 1 classes.


In [34]:
def getResult(output_path,y_pred):
    import os
    import pandas as pd
    
    path = './data/test_A/test'
    filename = []
    for file in os.listdir(path):
        filename.append(file)

    f = []
    for s in filename:
        for i in range(30):
            f.append(s.split('.')[0])

    index = [i for i in range(1,31)]*len(filename)

    y_pred = y_pred.clip(min=0.003, max=0.995)

    import numpy as np
    def softmax(x):
        """Compute the softmax of vector x."""
        exp_x = np.exp(x)
        softmax_x = exp_x / np.sum(exp_x)
        return softmax_x

    for y in range(len(y_pred)):
        y_pred[y] = softmax(y_pred[y])


    prob = [list(y_pred[j]) for j in range(len(y_pred))]
    probs = []
    for p in prob:
        probs.extend(p)

    pd.DataFrame({'f':f,'index':index,'probs':probs}).to_csv(output_path,header=None,index=None)    
    print(pd.read_csv(output_path,header=None))

    
getResult('result/incepV3_e50_finetune.csv',result)

          0   1         2
0      3108   1  0.031549
1      3108   2  0.031549
2      3108   3  0.031549
3      3108   4  0.031549
4      3108   5  0.031549
5      3108   6  0.031549
6      3108   7  0.031549
7      3108   8  0.031549
8      3108   9  0.031549
9      3108  10  0.031549
10     3108  11  0.031549
11     3108  12  0.031549
12     3108  13  0.031549
13     3108  14  0.031549
14     3108  15  0.031549
15     3108  16  0.085076
16     3108  17  0.031549
17     3108  18  0.031549
18     3108  19  0.031549
19     3108  20  0.031549
20     3108  21  0.031549
21     3108  22  0.031549
22     3108  23  0.031549
23     3108  24  0.031549
24     3108  25  0.031549
25     3108  26  0.031549
26     3108  27  0.031549
27     3108  28  0.031549
28     3108  29  0.031549
29     3108  30  0.031549
...     ...  ..       ...
89970  4593   1  0.031549
89971  4593   2  0.031549
89972  4593   3  0.031549
89973  4593   4  0.031549
89974  4593   5  0.031549
89975  4593   6  0.031549
89976  4593 

In [36]:
import pandas as pd
a = pd.read_csv('result/incepV3_e50_finetune.csv',header=None)
max(a.iloc[:,2])

0.08507601916789999

In [ ]:
# 记录
# 不去先训练全连接层的情况下直接去微调后几层，效果还不错，还得试一试先训练50个epoch再去微调的效果。
# 